In [1]:
import os
if os.path.exists("/gss_gpfs_scratch/"):
    RMG_PY_PATH = os.path.expanduser('/gss_gpfs_scratch/westgroup/Importer/RMG-Py/')
    RMG_MODELS_PATH = os.path.expanduser('/gss_gpfs_scratch/westgroup/Importer/RMG-models')
else:
    RMG_PY_PATH = os.path.expanduser('~/Code/RMG-Py/')
    RMG_MODELS_PATH = os.path.expanduser('~/Code/RMG-models')

In [2]:
import IPython
from IPython.display import display
import sys
import os
import re

sys.path.insert(1,RMG_PY_PATH) # a copy of RMG-Py on the `importer` branch.
from rmgpy.molecule import Molecule
import rmgpy.kinetics
import numpy
import cPickle as pickle
from collections import Counter, defaultdict
from rmgpy.reaction import Reaction
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import rmgpy.rmg
import rmgpy.data
import rmgpy.data.kinetics
import ck2cti
import numpy as np

# This is an ipython notebook to process the results from running `sarathy_comparison.py` and `heptane_comparison.py`

### This is importing the ignition delays

In [3]:
f = open("../reference_files/ignition_delay_heptane.pkl", "r")
heptane_data = pickle.load(f)

g = open("../reference_files/ignition_delay_sarathy.pkl", "r")
sarathy_data = pickle.load(g)

### This is getting the species dictionaries for each model

In [4]:
master = 'CombFlame2012/2028-Sarathy'
# Find and read the chemkin file
with open(os.path.join(RMG_MODELS_PATH, master,'import.sh')) as infile:
    shellscript = infile.read()
reactions_filename = re.search('--reactions\s+(\S+)',shellscript).group(1)
reactions_filepath = os.path.join(RMG_MODELS_PATH,master,reactions_filename)
thermo_filename = re.search('--thermo\s+(\S+)',shellscript).group(1)
thermo_filepath = os.path.join(RMG_MODELS_PATH,master,thermo_filename)
print(reactions_filepath)
print(thermo_filepath)
with open(reactions_filepath) as infile:
    chemkin = infile.readlines()
#for i,line in enumerate(chemkin):
    #print i, line.strip()             # uncomment to print the chemkin model
    
print "".join(chemkin[:4]) # print first 4 lines only

sarathy_parser = ck2cti.Parser()
surfaces = sarathy_parser.convertMech(inputFile=reactions_filepath,
                              thermoFile=thermo_filepath,
                              transportFile=None,
                              surfaceFile=None,
                              phaseName=None,
                              outName='sarathy_master.cti',
                              permissive=True)
sarathy_parser.reactions[0]

dict_path = os.path.join(RMG_MODELS_PATH, master, 'RMG-Py-kinetics-library', 'dictionary.txt')
print "Loading species_dict from",dict_path
sarathy_dict = rmgpy.data.kinetics.KineticsLibrary().getSpecies(dict_path)

oh*<=>oh+hv                                  1.450e+06      0.0           0.0
If the "--permissive" option was specified, this will be converted to an irreversible reaction with the photon removed.
ch*<=>ch+hv 1.860e+06 0.0 0.0
If the "--permissive" option was specified, this will be converted to an irreversible reaction with the photon removed.


/Users/nathan/Code/RMG-models/CombFlame2012/2028-Sarathy/model.txt
/Users/nathan/Code/RMG-models/CombFlame2012/2028-Sarathy/thermo.txt
!A comprehensive chemical kinetic combustion model for the four butanol isomers
!S.M. Sarathy, S. Vranckx, K. Yasunaga, M. Mehl, P. O�wald, W.K. Metcalfe,
!C. K. Westbrook, W.J. Pitz, K. Kohse-Hoinghaus, R.X. Fernandes, H.J Curran
!  Accepted Combustion and Flame, Dec 2011



INFO:root:Skipping unexpected species "hoco" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "ch3cho2h" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "ch3coch2o2h" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "ch3coch2o" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "ch3chcho" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "c3h2" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "c5h11-1" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "c5h11-2" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "c5h11-3" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "c5h10-1" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "c5h10-2" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "c5h81-3" while reading thermodynami

Wrote CTI mechanism file to 'sarathy_master.cti'.
Mechanism contains 431 species and 2346 reactions.
Loading species_dict from /Users/nathan/Code/RMG-models/CombFlame2012/2028-Sarathy/RMG-Py-kinetics-library/dictionary.txt


In [5]:
master = 'n-Heptane'
# Find and read the chemkin file
with open(os.path.join(RMG_MODELS_PATH, master,'import.sh')) as infile:
    shellscript = infile.read()
reactions_filename = re.search('--reactions\s+(\S+)',shellscript).group(1)
reactions_filepath = os.path.join(RMG_MODELS_PATH,master,reactions_filename)
thermo_filename = re.search('--thermo\s+(\S+)',shellscript).group(1)
thermo_filepath = os.path.join(RMG_MODELS_PATH,master,thermo_filename)
print(reactions_filepath)
print(thermo_filepath)
with open(reactions_filepath) as infile:
    chemkin = infile.readlines()
#for i,line in enumerate(chemkin):
    #print i, line.strip()             # uncomment to print the chemkin model
    
print "".join(chemkin[:4]) # print first 4 lines only

heptane_parser = ck2cti.Parser()
surfaces = heptane_parser.convertMech(inputFile=reactions_filepath,
                              thermoFile=thermo_filepath,
                              transportFile=None,
                              surfaceFile=None,
                              phaseName=None,
                              outName='sarathy_master.cti',
                              permissive=True)


dict_path = os.path.join(RMG_MODELS_PATH, master, 'RMG-Py-kinetics-library', 'dictionary.txt')
print "Loading species_dict from",dict_path
heptane_dict = rmgpy.data.kinetics.KineticsLibrary().getSpecies(dict_path)


/Users/nathan/Code/RMG-models/n-Heptane/nc7_ver3.1_mech.txt
/Users/nathan/Code/RMG-models/n-Heptane/n_heptane_v3.1_therm.dat
!n-heptane mechanism ver. 3.1 2012-03-30
!M. Mehl, W. J. Pitz, C. K. Westbrook and H. J. Curran, "Kinetic Modeling of Gasoline Surrogate Components and Mixtures under Engine Conditions," Proc. Combust. Inst.  33 (1) (2011) 193-200.
!LLNL-MI-536391
!March 2012: Multiplied the A-factors of the molecular elimination of HO2 from heptyl-O2 by a factor of 2. The previous version reported lower values doe to a clerical error in the uploaded file.



INFO:root:Skipping unexpected species "HOCO" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "CH3CHO2H" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "CH3CHCHO" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "C3H51-2V3OOH" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "C3H52-1V3OOH" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "IC5H12" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "AC5H11" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "BC5H11" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "CC5H11" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "DC5H11" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "AC5H10" while reading thermodynamics entry.
INFO:root:Skipping unexpected species "BC5H10" while reading thermodynami

Wrote CTI mechanism file to 'sarathy_master.cti'.
Mechanism contains 654 species and 4846 reactions.
Loading species_dict from /Users/nathan/Code/RMG-models/n-Heptane/RMG-Py-kinetics-library/dictionary.txt


In [6]:
from scipy.interpolate import UnivariateSpline

def estimate_std_dev(indep_variable, dep_variable):
    """

    Parameters
    ----------
    indep_variable : ndarray, list(float)
        Independent variable (e.g., temperature, pressure)
    dep_variable : ndarray, list(float)
        Dependent variable (e.g., ignition delay)

    Returns
    -------
    standard_dev : float
        Standard deviation of difference between data and best-fit line

    """

    assert len(indep_variable) == len(dep_variable), \
        'independent and dependent variables not the same length'

    # ensure data sorted based on independent variable to avoid some problems
    sorted_vars = sorted(zip(indep_variable, dep_variable))
    indep_variable = [pt[0] for pt in sorted_vars]
    dep_variable = [pt[1] for pt in sorted_vars]

    # spline fit of the data
    if len(indep_variable) == 1 or len(indep_variable) == 2:
        # Fit of data will be perfect
        return min_deviation
    elif len(indep_variable) == 3:
        spline = UnivariateSpline(indep_variable, dep_variable, k=2)
    else:
        spline = UnivariateSpline(indep_variable, dep_variable)

    standard_dev = numpy.std(dep_variable - spline(indep_variable))

    if standard_dev < 0.1:
        print('Standard deviation of {:.2f} too low, '
              'using {:.2f}'.format(standard_dev, 0.1))
        standard_dev = 0.1

    return standard_dev



In [7]:
print "These are the errors for Sarathy model swaps"
from copy import deepcopy
test_data = deepcopy(sarathy_data)
test_data = test_data[test_data.apply(np.isfinite)].apply(np.log10)
data = test_data["0"][test_data['0'].apply(np.isfinite)]
temps = test_data["0"][test_data['0'].apply(np.isfinite)].index

standard_deviation = estimate_std_dev(temps, data)

sarathy_results = []
for col in test_data.columns:
    
    if col == "0":
        continue 
        
    variation_data = test_data[col]
    print sarathy_parser.reactions[int(col)]
    
    i = 0
    E_sum = 0.0
    
    for temp in temps:
        log_delay = test_data[col].loc[temp]
        sarathy_delay = test_data['0'].loc[temp]
        i +=1
        E_sum += ((sarathy_delay - log_delay) / standard_deviation)**2
        
        i +=1
        
    E_sum = E_sum / float(i)
    
    sarathy_results.append([col, E_sum])
    
        
    print "For reaction {0} the error is {1}".format(col, E_sum)
    
    print

These are the errors for Sarathy model swaps
Standard deviation of 0.01 too low, using 0.10
c2h5coch3 + o <=> ch2ch2coch3 + oh
For reaction 1150 the error is 2.03818559265e-06

ic4h8oh-2o2 + h2o2 <=> ic4h8oh-2o2h + ho2
For reaction 2105 the error is 1.02337273371e-06

nc4h9oh + ch3 <=> c4h8oh-4 + ch4
For reaction 1568 the error is 4.77658778352e-06

nc3h7cho + ho2 <=> c3h6cho-2 + h2o2
For reaction 1194 the error is 7.13462540004e-07

nc3h7cho + ch3 <=> nc3h7co + ch4
For reaction 1187 the error is 0.000187754724305

c2h6 + ch3o2 <=> c2h5 + ch3o2h
For reaction 196 the error is 4.63734778901e-06

ch2o + oh <=> hoch2o
For reaction 77 the error is 0.017564110752

c2h5oh + ho2 <=> pc2h4oh + h2o2
For reaction 369 the error is 1.49503894172e-06

ch3och3 + ch3o2 <=> ch3och2 + ch3o2h
For reaction 435 the error is 1.0543475638e-06

ic4h8oh-3o2 + h2o2 <=> ic4h8oh-3o2h + ho2
For reaction 2106 the error is 9.30657870328e-07

pc4h9o2 + pc4h9o2 -> o2 + pc4h9o + pc4h9o
For reaction 1081 the error is 6.

In [8]:
sarathy_results = pd.DataFrame(sarathy_results)
sarathy_results.colums = ["Reaction", "Error"]
sarathy_mean = sarathy_results[sarathy_results[1].apply(np.isfinite)][1].mean()
sarathy_std = sarathy_results[sarathy_results[1].apply(np.isfinite)][1].std()

sarathy_funct = lambda x: abs(x - sarathy_mean) / sarathy_std

In [9]:
print "These are the errors for Sarathy model swaps"
from copy import deepcopy
test_data = deepcopy(heptane_data)
test_data = test_data[test_data.apply(np.isfinite)].apply(np.log10)
data = test_data["0"][test_data['0'].apply(np.isfinite)]
temps = test_data["0"][test_data['0'].apply(np.isfinite)].index

standard_deviation = estimate_std_dev(temps, data)

heptane_results = []
for col in test_data.columns:
    
    if col == "0":
        continue 
        
    variation_data = test_data[col]
    
    i = 0
    E_sum = 0.0
    
    for temp in temps:
        log_delay = test_data[col].loc[temp]
        heptane_delay = test_data['0'].loc[temp]
        i +=1
        E_sum += ((heptane_delay - log_delay) / standard_deviation)**2
        
        i +=1
        
    E_sum = E_sum / float(i)
    
    heptane_results.append([col, E_sum])
    
        
    print "For reaction {0} the error is {1}".format(col, E_sum)
    
    print

These are the errors for Sarathy model swaps
For reaction 1285 the error is 1.18451946468e-08

For reaction 2637 the error is 2.30445708936e-09

For reaction 3486 the error is 3.56494147562e-10

For reaction 2493 the error is 1.02477175785e-09

For reaction 2195 the error is 9.12565049121e-10

For reaction 3573 the error is 7.03014137027e-11

For reaction 3621 the error is 2.08572900754e-05

For reaction 2591 the error is 2.36701246671e-07

For reaction 285 the error is 1.17622142057e-06

For reaction 105 the error is 1.53364555109e-07

For reaction 2957 the error is 4.48806157957e-10

For reaction 4350 the error is 5.09786627563e-10

For reaction 33 the error is 6.77600873193e-06

For reaction 2062 the error is 1.68062015695e-10

For reaction 2494 the error is 1.08280363517e-09

For reaction 2760 the error is 1.00078726734e-09

For reaction 86 the error is 0.000690142256055

For reaction 1726 the error is 1.21414151712e-09

For reaction 4112 the error is 7.39843864079e-08

For reactio

In [10]:
heptane_results = pd.DataFrame(heptane_results)
heptane_results.colums = ["Reaction", "Error"]
heptane_mean = heptane_results[heptane_results[1].apply(np.isfinite)][1].mean()
heptane_std = heptane_results[heptane_results[1].apply(np.isfinite)][1].std()

heptane_funct = lambda x: abs(x - heptane_mean) / heptane_std

In [24]:
for i in sarathy_results[0][sarathy_results[1].apply(sarathy_funct) > 3]:
    print i

77


### Printing out the indicies of the significant reactions

In [29]:

important_sarathy_reactions = {}
print "Sarathy significant reactions:"
for reaction_index in sarathy_results[0][sarathy_results[1].apply(sarathy_funct) > 3]: 
    print reaction_index
    try:
        rxn = sarathy_parser.reactions[int(reaction_index)+20]
        print rxn
        r1, r2 = rxn.reactants
        p1, p2 = rxn.products
        s1 = r1[1]
        s2 = r2[1]
        s3 = p1[1]
        s4 = p2[1]
        print "Made it this far"
        
        important_sarathy_reactions[reaction_index] = Reaction(
            reactants = [sarathy_dict[s1.label], sarathy_dict[s2.label]], 
            products= [sarathy_dict[s3.label], sarathy_dict[s4.label]],
            reversible = True)
    except:
        continue
    
print
print "#####"
print 
important_heptane_reactions = {}

print "Heptane significant reactions:"
for reaction_index in heptane_results[0][heptane_results[1].apply(heptane_funct) > 3]:
    print reaction_index
    try:
        rxn = heptane_parser.reactions[int(reaction_index) + 1]
        print rxn
        r1, r2 = rxn.reactants
        p1, p2 = rxn.products
        s1 = r1[1]
        s2 = r2[1]
        s3 = p1[1]
        s4 = p2[1]

        important_heptane_reactions[reaction_index] = Reaction(
            reactants = [heptane_dict[s1.label], heptane_dict[s2.label]], 
            products= [heptane_dict[s3.label], heptane_dict[s4.label]],
            reversible = True) 
    except:
        print "Issue converting reaction {} into RMG reaction.".format(reaction_index)
        continue

Sarathy significant reactions:
77
ch3o + ho2 <=> ch2o + h2o2
Made it this far

#####

Heptane significant reactions:
3624
NC7H16 + HO2 -> C7H15-4 + H2O2
3622
NC7H16 + HO2 -> C7H15-3 + H2O2


In [30]:



# Viewing the significant reactions for both models
important_sarathy_reactions, important_heptane_reactions

({'77': Reaction(reactants=[Species(label="ch3o", molecule=[Molecule(SMILES="C[O]")]), Species(label="ho2", molecule=[Molecule(SMILES="[O]O")])], products=[Species(label="ch2o", molecule=[Molecule(SMILES="C=O")]), Species(label="h2o2", molecule=[Molecule(SMILES="OO")])])},
 {'3622': Reaction(reactants=[Species(label="NC7H16", molecule=[Molecule(SMILES="CCCCCCC")]), Species(label="HO2", molecule=[Molecule(SMILES="[O]O")])], products=[Species(label="C7H15-3", molecule=[Molecule(SMILES="CC[CH]CCCC")]), Species(label="H2O2", molecule=[Molecule(SMILES="OO")])]),
  '3624': Reaction(reactants=[Species(label="NC7H16", molecule=[Molecule(SMILES="CCCCCCC")]), Species(label="HO2", molecule=[Molecule(SMILES="[O]O")])], products=[Species(label="C7H15-4", molecule=[Molecule(SMILES="CCC[CH]CCC")]), Species(label="H2O2", molecule=[Molecule(SMILES="OO")])])})

### Importing the importer kinetics (need to remake this pickle file)

In [31]:
h = open("../reference_files/importerKinetics.pkl", "r")
importer_kinetics = pickle.load(h)
importer_kinetics = pd.DataFrame(importer_kinetics)
importer_kinetics

,ic4h7 + ho2 <=> ic4h7o + oh,c4h612 + h <=> c3h4-p + ch3,c4h3-i + h <=> c4h2 + h2,c4h3-i + oh <=> c4h2 + h2o,CH2CCH3 + CH2CO => C3H6 + HCCO,ch3chchcho <=> c3h6 + co,H2CCCH + OH <=> HCO + C2H3,c2h5coch3 + h <=> c2h5coch2 + h2,c4h3-i + ch2 <=> c3h4-a + c2h,H2CCCH + CH3 <=> C4H6,...,tc4h8cho + o2 <=> o2c4h8cho,C3H3 + CH <=> C4H2 + H + H,HCO3 + C3H6 => HCO3H + CH2CHCH2,o2c4h8cho <=> o2hc4h8co,ic4h8o2h-t + co <=> o2hc4h8co,c3h5-a + c2h5 <=> c2h4 + c3h6,C2H5OO + CO => CO2 + CH3 + CH2O,c2h5coch3 + c2h5 <=> c2h5coch2 + c2h6,ic4h7o + ic4h8 <=> ic4h7oh + ic4h7,ic4h6oh + ho2 => ch2cch2oh + ch2o + oh
AramcoMech_1.3,"Arrhenius(A=(7e+12,'cm^3/(mol*s)'), n=0, Ea=(-...","Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(2...","Arrhenius(A=(6e+13,'cm^3/(mol*s)'), n=0, Ea=(0...","Arrhenius(A=(4e+12,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(3.9e+14,'s^-1'), n=0, Ea=(69000,'...",NaN,"Arrhenius(A=(9.3e+12,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,...,"Arrhenius(A=(2e+12,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,NaN,"Arrhenius(A=(2.16e+11,'s^-1'), n=0, Ea=(15360,...","Arrhenius(A=(1.5e+11,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(4e+11,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(5e+10,'cm^3/(mol*s)'), n=0, Ea=(1...","Arrhenius(A=(2.7e+11,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(1.446e+13,'cm^3/(mol*s)'), n=0, E..."
AramcoMech_2.0,"PDepArrhenius(pressures=([0.01,0.1,1,10,100],'...","Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(2...","Arrhenius(A=(6e+13,'cm^3/(mol*s)'), n=0, Ea=(0...","Arrhenius(A=(4e+12,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(3.9e+14,'s^-1'), n=0, Ea=(69000,'...","Arrhenius(A=(1e+13,'cm^3/(mol*s)'), n=0, Ea=(0...","Arrhenius(A=(9.3e+12,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,...,"Arrhenius(A=(2e+12,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,NaN,"Arrhenius(A=(2.16e+11,'s^-1'), n=0, Ea=(15360,...","Arrhenius(A=(1.5e+11,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(4e+11,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(5e+10,'cm^3/(mol*s)'), n=0, Ea=(1...","Arrhenius(A=(2.7e+11,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(1.446e+13,'cm^3/(mol*s)'), n=0, E..."
AutoTST-OOHabstraction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Biomass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chernov,NaN,NaN,"Arrhenius(A=(5e+13,'cm^3/(mol*s)'), n=0, Ea=(0...","Arrhenius(A=(3e+13,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,NaN,"Arrhenius(A=(4e+13,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(0...","ThirdBody(arrheniusLow=Arrhenius(A=(2.6e+58,'c...",...,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(2.6e+12,'cm^3/(mol*s)'), n=0, Ea=...",NaN,NaN,NaN,NaN
CombFlame2012/2028-Sarathy,"Arrhenius(A=(7e+12,'cm^3/(mol*s)'), n=0, Ea=(-...","Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(2...","Arrhenius(A=(6e+13,'cm^3/(mol*s)'), n=0, Ea=(0...","Arrhenius(A=(4e+12,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(3.9e+14,'s^-1'), n=0, Ea=(69000,'...",NaN,"Arrhenius(A=(9.3e+12,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,...,"Arrhenius(A=(2e+12,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,NaN,"Arrhenius(A=(2.16e+11,'s^-1'), n=0, Ea=(15360,...","Arrhenius(A=(1.5e+11,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(4e+11,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(5e+10,'cm^3/(mol*s)'), n=0, Ea=(1...","Arrhenius(A=(2.7e+11,'cm^3/(mol*s)'), n=0, Ea=...","Arrhenius(A=(1.446e+13,'cm^3/(mol*s)'), n=0, E..."
CombFlame2013/1315-Chang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1541-Zhang,"Arrhenius(A=(7e+12,'cm^3/(mol*s)'), n=0, Ea=(-...",NaN,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(9.3e+12,'cm^3/(mol*s)'), n=0, Ea=...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(4e+11,'cm^3/(mol*s)'), n=0, Ea=(0...",NaN,"Arrhenius(A=(5e+10,'cm^3/(mol*s)'), n=0, Ea=(1...","Arrhenius(A=(2.7e+11,'cm^3/(mol*s)'), n=0, Ea=...

### Getting the list of autoTST OOH reactions

In [32]:
f = open("../../autotst_kinetics.pkl","r")
autotst_kinetics = pickle.load(f)
ooh_reactions = []
for rxn in autotst_kinetics:
    reactants, products = rxn.label.split("_")
    r1, r2 = reactants.split('+')
    p1, p2 = products.split('+')
    if "OO" in [r1, r2, p1, p2] and "[O]O" in [r1, r2, p1, p2]:
        #print [r1, r2, p1, p2]
        ooh_reactions.append(rxn)

### Creating nickname dictionaries 

In [33]:
sarathy_smiles_to_nickname_dict = {}
for species in sarathy_dict.itervalues():
    #print len(species.molecule[0].toSMILES())
    for mol in species.molecule:
        sarathy_smiles_to_nickname_dict[mol.toSMILES()] = species.label
    

heptane_smiles_to_nickname_dict = {}
for species in heptane_dict.itervalues():
    #print len(species.molecule[0].toSMILES())
    for mol in species.molecule:
        heptane_smiles_to_nickname_dict[mol.toSMILES()] = species.label
    
print "Dictionaries created"

Dictionaries created


In [34]:
important_sarathy_reactions

{'77': Reaction(reactants=[Species(label="ch3o", molecule=[Molecule(SMILES="C[O]")]), Species(label="ho2", molecule=[Molecule(SMILES="[O]O")])], products=[Species(label="ch2o", molecule=[Molecule(SMILES="C=O")]), Species(label="h2o2", molecule=[Molecule(SMILES="OO")])])}

In [35]:
import itertools
sarathy_rxns = []
for sarathy_reaction in important_sarathy_reactions.values():
    
    reactants = [n.molecule[-1].toSMILES() for n in sarathy_reaction.reactants]
    products = [n.molecule[-1].toSMILES() for n in sarathy_reaction.products]

    joined_reactant_orders = ['+'.join(order) for order in itertools.permutations(reactants)]
    joined_product_orders = ['+'.join(order) for order in itertools.permutations(products)]
    possible_labels = ['_'.join((joined_r, joined_p)) for joined_r in joined_reactant_orders for joined_p in joined_product_orders]
    
    
    for ooh_reaction in ooh_reactions:
        if ooh_reaction.label in possible_labels:
            ooh_reactants, ooh_products = ooh_reaction.label.split("_")
            r1, r2 = ooh_reactants.split("+")
            p1, p2 = ooh_products.split("+")
            ooh_smiles = [r1, r2, p1, p2]

            inchikey_to_smiles_dict = {}
            for smiles in ooh_smiles:
                inchikey_to_smiles_dict[Molecule(SMILES=smiles).toInChIKey()] = smiles
        
    
            for reactant in ooh_reaction.reactants:
                inchi_key = reactant.label.split("-u")[0]
                if not reactant.label in sarathy_smiles_to_nickname_dict.itervalues():
                    reactant.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
                    reactant.label = sarathy_smiles_to_nickname_dict[inchikey_to_smiles_dict[inchi_key]]
                
                
            for product in ooh_reaction.products:
                inchi_key = product.label.split("-u")[0]
                if not product.label in sarathy_smiles_to_nickname_dict.itervalues():
                    product.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
                    product.label = sarathy_smiles_to_nickname_dict[inchikey_to_smiles_dict[inchi_key]]
                
                
            
            sarathy_rxns.append([sarathy_reaction, ooh_reaction])#, reaction.toChemkin(), reaction.toCantera()])
        
sarathy_df = pd.DataFrame(sarathy_rxns)

sarathy_df.columns = ["Importer Reaction", "AutoTST Reaction"]#, "AutoTST Reaction - Chemkin" , "AutoTST - Cantera"]

sarathy_df


ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [ ]:
sarathy_df

In [36]:
import itertools
heptane_rxns = []
for heptane_reaction in important_heptane_reactions.values():
    
    reactants = [n.molecule[-1].toSMILES() for n in heptane_reaction.reactants]
    products = [n.molecule[-1].toSMILES() for n in heptane_reaction.products]

    joined_reactant_orders = ['+'.join(order) for order in itertools.permutations(reactants)]
    joined_product_orders = ['+'.join(order) for order in itertools.permutations(products)]
    possible_labels = ['_'.join((joined_r, joined_p)) for joined_r in joined_reactant_orders for joined_p in joined_product_orders]
    
    
    for ooh_reaction in ooh_reactions:
        if ooh_reaction.label in possible_labels:
            ooh_reactants, ooh_products = ooh_reaction.label.split("_")
            r1, r2 = ooh_reactants.split("+")
            p1, p2 = ooh_products.split("+")
            ooh_smiles = [r1, r2, p1, p2]

            inchikey_to_smiles_dict = {}
            for smiles in ooh_smiles:
                inchikey_to_smiles_dict[Molecule(SMILES=smiles).toInChIKey()] = smiles
        
    
            for reactant in ooh_reaction.reactants:
                inchi_key = reactant.label.split("-u")[0]
                if not reactant.label in heptane_smiles_to_nickname_dict.itervalues():
                    reactant.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
                    reactant.label = heptane_smiles_to_nickname_dict[inchikey_to_smiles_dict[inchi_key]]
                
                
            for product in ooh_reaction.products:
                inchi_key = product.label.split("-u")[0]
                if not product.label in heptane_smiles_to_nickname_dict.itervalues():
                    product.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
                    product.label = heptane_smiles_to_nickname_dict[inchikey_to_smiles_dict[inchi_key]]
                
                
            
            heptane_rxns.append([heptane_reaction, ooh_reaction])#, reaction.toChemkin(), reaction.toCantera()])
        
heptane_df = pd.DataFrame(heptane_rxns)

heptane_df.columns = ["Importer Reaction", "AutoTST Reaction"]#, "AutoTST Reaction - Chemkin" , "AutoTST - Cantera"]

heptane_df


,Importer Reaction,AutoTST Reaction
0,NC7H16 + HO2 <=> C7H15-3 + H2O2,HO2 + NC7H16 <=> H2O2 + C7H15-3
1,NC7H16 + HO2 <=> C7H15-4 + H2O2,HO2 + NC7H16 <=> C7H15-4 + H2O2


In [37]:
reactions_df = pd.concat([sarathy_df, heptane_df])
reactions_df

,Importer Reaction,AutoTST Reaction
0,NC7H16 + HO2 <=> C7H15-3 + H2O2,HO2 + NC7H16 <=> H2O2 + C7H15-3
1,NC7H16 + HO2 <=> C7H15-4 + H2O2,HO2 + NC7H16 <=> C7H15-4 + H2O2


### Plotting the results

In [ ]:
import numpy as np
inverseTemps = np.linspace(1000./500., 1000./2500., 15)
Temps = 1000./inverseTemps

comparisonPressure = 1e5 # Pa
for autotst_rxn in reactions_df["AutoTST Reaction"]:
        
    for i, reaction in enumerate(importer_kinetics.columns):
        if reaction.isIsomorphic(autotst_rxn):
            rxn_kinetics = importer_kinetics.iloc[:,i].dropna()

            rxn_kinetics["AutoTST"] = autotst_rxn.kinetics
            
            
            print rxn_kinetics
            print 
            
            
            fig, ax = plt.subplots()
            for index in rxn_kinetics.index:
                logk = []
                logkAutoTST = []
                kinetics = rxn_kinetics[index]

                if index == 'AutoTST':

                    for Temp in Temps:
                        k = kinetics.getRateCoefficient(T=Temp,P=comparisonPressure)
                        logkAutoTST.append(np.log10(k) + 6)

                    plt.plot(inverseTemps, logkAutoTST, '-r', linewidth=2)
                else:
                    for Temp in Temps:
                        k = kinetics.getRateCoefficient(T=Temp,P=comparisonPressure)
                        logk.append(np.log10(k) + 6)
                    plt.plot(inverseTemps, logk, ':k', linewidth=2, alpha=0.4)
            plt.xlabel("$1/T [K^{-1}]$", fontsize=16)
            plt.ylabel("$log_{10}(k) [cm^3 / (mole \cdot s)]$", fontsize=16)
            label = autotst_rxn.label

            Tticks = [500, 1000, 2500]
            ax.set_xticks([1000./T for T in Tticks])
            ax.set_xticklabels(['1/{:.0f}'.format(T) for T in Tticks])
            plt.tick_params(axis='x', labelsize=11)
            plt.tick_params(axis='y', labelsize=11)
            #plt.title(label)
            plt.xlim([0.39,2])

            plt.ylim(max([ax.get_ylim()[0], 0]), min([ax.get_ylim()[1], 15]))
            saveString = os.path.join("../reference_files/", str(autotst_rxn) + '.pdf')
            plt.savefig(saveString)
            print str(label)
            plt.show() 
    
